In [16]:
import requests
import json
import pandas as pd

YELP_API_KEY = YELP_API_KEY
API_ENDPOINT = 'https://api.yelp.com/v3/businesses/search'

# Yelp API endpoint URL for exploring restaurants
yelp_url = 'https://api.yelp.com/v3/businesses/search'
yelp_headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}

venues_data_list = []

yelp_params = {
    'location': 'Castleridge,Calgary',
    'radius': 2000,  # Radius for restaurant search (adjust as needed)
    'categories': 'restaurants',
    'limit': 50,  # Number of restaurants to retrieve (adjust as needed)
    }

yelp_response = requests.get(API_ENDPOINT, params=yelp_params, headers=yelp_headers)

# Check if the Yelp response is successful
if yelp_response.status_code == 200:
    yelp_data = json.loads(yelp_response.text)
    for business in yelp_data.get('businesses', []):
        categories = [category.get('title') for category in business.get('categories', [])]
        cuisine_keywords = ['coffee','fast food', 'burgers', 'sushi', 'pizza', 'sandwiches', 'pub', 'bar','cafe','steak','seafood','dessert','beer']
        nationality_keywords = ['european','greek','african','mexican','venezuelan','vietnamese','malaysian','caribbean','persian','filipino','irish','british','scottish','ethiopian','american', 'japanese', 'mediterranean', 'thai', 'french', 'italian', 'middle eastern','canadian','asian','indian','chinese','german','korean','peruvian']
        cuisine_list = []
        nationality_list = []
        for category in categories:
            category_lower = category.lower()
            if any(keyword in category_lower for keyword in cuisine_keywords):
                cuisine_list.append(category)
            elif any(keyword in category_lower for keyword in nationality_keywords):
                nationality_list.append(category)
        restaurant_details = {
            'Name': business.get('name'),
            'Categories': ', '.join(categories),
            'Cuisine Types': ', '.join(cuisine_list),
            'Nationalities': ', '.join(nationality_list),
            'Rating': business.get('rating'),
            'Address': ', '.join([business.get('location', {}).get('address1'), business.get('location', {}).get('city')]) or 'N/A',
            'Res_Latitude': business.get('coordinates', {}).get('latitude'),
            'Res_Longitude': business.get('coordinates', {}).get('longitude'),
            'Reviews': business.get('review_count'),
            'Distance': business.get('distance'),
            'Phone': business.get('phone') or 'N/A',
            'Closed': business.get('is_closed'),
            'Price': business.get('price') or 'N/A',
        }
        venues_data_list.append(restaurant_details)

# Create a DataFrame from the restaurant data list
Castleridge = pd.DataFrame(venues_data_list)


Castleridge.shape

(50, 13)

In [17]:
Calgary_df = pd.concat([Inglewood,
Stampede,
Olympic_Park,
Heritage_Park,
Sunridge,
Thorncliffe,
Hillhurst,
Glenbrook,
Dover,
Acadia,
Varsity,
Castleridge],axis=0)

In [18]:
Calgary_df.shape

(542, 13)

In [19]:
Calgary_no_duplicates = Calgary_df.drop_duplicates(subset=['Name', 'Categories', 'Cuisine Types', 'Nationalities','Address'])

In [20]:
Calgary_no_duplicates.shape

(538, 13)

In [21]:
Calgary_no_duplicates.head()

,Name,Categories,Cuisine Types,Nationalities,Rating,Address,Res_Latitude,Res_Longitude,Reviews,Distance,Phone,Closed,Price
0,Blackfoot Truckstop Diner,"Diners, Breakfast & Brunch",,,3.5,"1840 9 Avenue SE, Calgary",51.036434,-114.020316,84,183.167899,+14032655964,False,$$
1,The Blues Can,"Jazz & Blues, Bars, Cajun/Creole",Bars,,4.0,"1429 9 Avenue SE, Calgary",51.039250,-114.029990,42,654.231270,+14032622666,False,$$
2,The Nash,"Bars, Canadian (New)",Bars,Canadian (New),4.0,"925 11th Street SE, Calgary",51.041415,-114.037001,145,1190.292526,+14039843365,False,$$$
3,The Eden,"French, Bistros",,French,5.0,"1219 9 Avenue SE, Calgary",51.041501,-114.035308,26,1119.749722,+14032661005,False,$$
4,Madison's 12|12,American,,American,3.5,"1212 9 Avenue SE, Calgary",51.042130,-114.035890,58,1175.378470,+14034524970,False,$$


In [23]:
Calgary_no_duplicates.to_csv('../data/Calgary.csv', index=False) 

In [24]:
Calgary_df = pd.read_csv('../data/Calgary.csv')


In [25]:
Calgary_df['Price'].fillna(method='ffill', inplace=True)
columns_to_fill = ['Nationalities','Phone','Categories']
Calgary_df[columns_to_fill] = Calgary_df[columns_to_fill].fillna('NA')


import pandas as pd
import numpy as np
def fill_cuisine(row):
    if pd.notnull(row['Cuisine Types']):
        return row['Cuisine Types']
    categories = row['Categories'].split(', ')
    if len(categories) > 0:
        return categories[0]
    return np.nan

# Apply the function to fill 'Cuisine Types'
Calgary_df['Cuisine Types'] = Calgary_df.apply(fill_cuisine, axis=1)

Calgary_df['Phone'] = pd.to_numeric(Calgary_df['Phone'], errors='coerce').fillna(np.nan).astype('Int64').astype(str)

Calgary_df['City']='Calgary'
# Print the updated DataFrame
Calgary_df.head()

,Name,Categories,Cuisine Types,Nationalities,Rating,Address,Res_Latitude,Res_Longitude,Reviews,Distance,Phone,Closed,Price,City
0,Blackfoot Truckstop Diner,"Diners, Breakfast & Brunch",Diners,NA,3.5,"1840 9 Avenue SE, Calgary",51.036434,-114.020316,84,183.167899,14032655964,False,$$,Calgary
1,The Blues Can,"Jazz & Blues, Bars, Cajun/Creole",Bars,NA,4.0,"1429 9 Avenue SE, Calgary",51.039250,-114.029990,42,654.231270,14032622666,False,$$,Calgary
2,The Nash,"Bars, Canadian (New)",Bars,Canadian (New),4.0,"925 11th Street SE, Calgary",51.041415,-114.037001,145,1190.292526,14039843365,False,$$$,Calgary
3,The Eden,"French, Bistros",French,French,5.0,"1219 9 Avenue SE, Calgary",51.041501,-114.035308,26,1119.749722,14032661005,False,$$,Calgary
4,Madison's 12|12,American,American,American,3.5,"1212 9 Avenue SE, Calgary",51.042130,-114.035890,58,1175.378470,14034524970,False,$$,Calgary


In [26]:
print(Calgary_df.isnull().sum())

Name             0
Categories       0
Cuisine Types    0
Nationalities    0
Rating           0
Address          0
Res_Latitude     0
Res_Longitude    0
Reviews          0
Distance         0
Phone            0
Closed           0
Price            0
City             0
dtype: int64


In [30]:
Calgary_df.to_csv('../data/Calgary.csv', index=False) 

In [28]:
Calgary_df['Res_Latitude'] = Calgary_df['Res_Latitude'].round(6)
Calgary_df['Res_Longitude'] = Calgary_df['Res_Longitude'].round(6)
Calgary_df['Distance']=Calgary_df['Distance'].round(2)

In [29]:
Calgary_df.head()

,Name,Categories,Cuisine Types,Nationalities,Rating,Address,Res_Latitude,Res_Longitude,Reviews,Distance,Phone,Closed,Price,City
0,Blackfoot Truckstop Diner,"Diners, Breakfast & Brunch",Diners,NA,3.5,"1840 9 Avenue SE, Calgary",51.036434,-114.020316,84,183.17,14032655964,False,$$,Calgary
1,The Blues Can,"Jazz & Blues, Bars, Cajun/Creole",Bars,NA,4.0,"1429 9 Avenue SE, Calgary",51.039250,-114.029990,42,654.23,14032622666,False,$$,Calgary
2,The Nash,"Bars, Canadian (New)",Bars,Canadian (New),4.0,"925 11th Street SE, Calgary",51.041415,-114.037001,145,1190.29,14039843365,False,$$$,Calgary
3,The Eden,"French, Bistros",French,French,5.0,"1219 9 Avenue SE, Calgary",51.041501,-114.035308,26,1119.75,14032661005,False,$$,Calgary
4,Madison's 12|12,American,American,American,3.5,"1212 9 Avenue SE, Calgary",51.042130,-114.035890,58,1175.38,14034524970,False,$$,Calgary


In [31]:
Calgary_df.to_excel('../data/Calgary.xlsx', index=False)  # Specify the output file name and disable index column